In [233]:
import pandas as pd

# Save filepath
matches_file_path = '../epl_final.csv'
# Read data
matches = pd.read_csv(matches_file_path)
matches = matches.dropna(axis=0)
matches.describe()

,FullTimeHomeGoals,FullTimeAwayGoals,HalfTimeHomeGoals,HalfTimeAwayGoals,HomeShots,AwayShots,HomeShotsOnTarget,AwayShotsOnTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards
count,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000
mean,1.535394,1.182623,0.688273,0.518763,13.617484,10.810661,5.973134,4.693710,6.040299,4.774733,11.276333,11.765672,1.468124,1.792431,0.062473,0.085288
std,1.305432,1.157414,0.835079,0.735357,5.356424,4.696501,3.267954,2.750045,3.110619,2.749541,3.750250,3.924179,1.217190,1.288481,0.253221,0.290183
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,10.000000,7.000000,4.000000,3.000000,4.000000,3.000000,9.000000,9.000000,1.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,13.000000,10.000000,6.000000,4.000000,6.000000,4.000000,11.000000,12.000000,1.000000,2.000000,0.000000,0.000000
75%,2.000000,2.000000,1.000000,1.000000,17.000000,14.000000,8.000000,6.000000,8.000000,6.000000,14.000000,14.000000,2.000000,3.000000,0.000000,0.000000
max,9.000000,9.000000,5.000000,5.000000,43.000000,37.000000,24.000000,20.000000,20.000000,19.000000,33.000000,29.000000,7.000000,9.000000,3.000000,2.000000


In [234]:
print(matches.head())
print(matches.columns)
print(matches.shape)

    Season   MatchDate  HomeTeam       AwayTeam  FullTimeHomeGoals  \
0  2000/01  2000-08-19  Charlton       Man City                  4   
1  2000/01  2000-08-19   Chelsea       West Ham                  4   
2  2000/01  2000-08-19  Coventry  Middlesbrough                  1   
3  2000/01  2000-08-19     Derby    Southampton                  2   
4  2000/01  2000-08-19     Leeds        Everton                  2   

   FullTimeAwayGoals FullTimeResult  HalfTimeHomeGoals  HalfTimeAwayGoals  \
0                  0              H                  2                  0   
1                  2              H                  1                  0   
2                  3              A                  1                  1   
3                  2              D                  1                  2   
4                  0              H                  2                  0   

  HalfTimeResult  ...  HomeShotsOnTarget  AwayShotsOnTarget  HomeCorners  \
0              H  ...                 14

In [235]:
# Extract year, month & day
matches['MatchDate'] = pd.to_datetime(matches['MatchDate'])
matches['Year'] = matches['MatchDate'].dt.year
matches['Month'] = matches['MatchDate'].dt.month
matches['Day'] = matches['MatchDate'].dt.day

In [236]:
y = matches.FullTimeResult
match_features = ['Year', 'Month', 'Day', 'HomeTeam', 'AwayTeam']
X = matches[match_features]

In [237]:
X.describe()

,Year,Month,Day
count,9380.000000,9380.000000,9380.000000
mean,2012.529104,6.733902,15.962793
std,7.213963,3.928129,9.088657
min,2000.000000,1.000000,1.000000
25%,2006.000000,3.000000,8.000000
50%,2013.000000,8.000000,16.000000
75%,2019.000000,10.000000,24.000000
max,2025.000000,12.000000,31.000000


In [238]:
X.head()

,Year,Month,Day,HomeTeam,AwayTeam
0,2000,8,19,Charlton,Man City
1,2000,8,19,Chelsea,West Ham
2,2000,8,19,Coventry,Middlesbrough
3,2000,8,19,Derby,Southampton
4,2000,8,19,Leeds,Everton


In [239]:
mask = X['Year'] <= 2022
X_train = X[mask]
X_test = X[~mask]
y_train = y[mask]
y_test = y[~mask]

In [240]:
X_train = pd.get_dummies(X_train, columns=['HomeTeam', 'AwayTeam'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['HomeTeam', 'AwayTeam'], drop_first=True)

# Make sure both datasets have same columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

In [241]:
from sklearn.preprocessing import StandardScaler

numeric_cols = ['Year', 'Month', 'Day']

scaler = StandardScaler()

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [242]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=16, max_iter=100000, tol=1e-6, class_weight='balanced')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred

array(['H', 'A', 'A', 'H', 'H', 'H', 'H', 'H', 'A', 'D', 'A', 'H', 'A',
       'A', 'H', 'A', 'H', 'A', 'A', 'H', 'H', 'H', 'A', 'A', 'H', 'A',
       'H', 'D', 'D', 'A', 'A', 'A', 'H', 'A', 'A', 'H', 'A', 'A', 'H',
       'D', 'H', 'A', 'H', 'A', 'A', 'H', 'A', 'H', 'A', 'H', 'A', 'D',
       'A', 'A', 'H', 'H', 'A', 'A', 'D', 'D', 'H', 'A', 'A', 'A', 'A',
       'H', 'H', 'A', 'D', 'A', 'A', 'H', 'A', 'A', 'A', 'H', 'H', 'H',
       'H', 'A', 'D', 'H', 'A', 'A', 'A', 'D', 'D', 'A', 'D', 'D', 'A',
       'H', 'A', 'A', 'H', 'A', 'H', 'D', 'D', 'H', 'H', 'A', 'A', 'A',
       'H', 'H', 'A', 'H', 'D', 'D', 'A', 'H', 'H', 'A', 'A', 'A', 'D',
       'H', 'A', 'H', 'H', 'A', 'H', 'H', 'H', 'A', 'H', 'H', 'A', 'A',
       'A', 'H', 'A', 'H', 'H', 'D', 'H', 'A', 'H', 'A', 'A', 'A', 'H',
       'A', 'D', 'A', 'D', 'H', 'A', 'A', 'A', 'D', 'A', 'D', 'H', 'A',
       'A', 'H', 'H', 'A', 'A', 'H', 'D', 'A', 'A', 'H', 'D', 'H', 'D',
       'A', 'H', 'H', 'A', 'A', 'H', 'H', 'A', 'H', 'A', 'A', 'A

In [243]:
from sklearn.metrics import classification_report

def score_model(y_test, y_pred):
    print(classification_report(y_test, y_pred))

In [244]:
score_model(y_test, y_pred)

              precision    recall  f1-score   support

           A       0.39      0.59      0.47       300
           D       0.32      0.19      0.24       220
           H       0.57      0.49      0.53       426

    accuracy                           0.45       946
   macro avg       0.43      0.42      0.41       946
weighted avg       0.46      0.45      0.44       946

